In [1]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import zipfile
from pycocotools.coco import COCO
import os
import shutil
from PIL import Image
import tensorflow as tf
from tqdm.notebook import tqdm  # Import tqdm from tqdm.notebook for Jupyter compatibility
import json

# Self created notebooks for pre-processing coco datasets
import downloading_coco_datasets.ipynb
import filter_coco_datasets.ipynb
import resize_annotations.ipynb
import resize_coco_datasets.ipynb